In [1]:
prompt_original = {
    'quantitative_value':{
        'brightness':[],
        'structure':[],
        'parallel':[],
    },
    'single_select':{
        'line':[['curve line',1],['angular line',1],['straight line',1]],
        #'weight_center_location':[['upper left',1],['bottom left and upper right',1],['bottom right',1]],
        'weights':{
            'line':1,
        }
       
    },
    'multiple_select':{
        'hue':[['white',1],['grey',1],['black',1],['orange',1],['yellow',1],['red',1],['green',1],['blue',1],['violet',1]],#3/9
        'plane':[['point',1],['triangle',1],['square',1],['circle',1]],#2/4
        'multi_num':{
            'hue':3,
            'plane':2,
        },
        'weights':{
            'hue':1,
            'plane':1,
        }

    },
    
}

types = ['single_select','multiple_select']
s_attributes = [key for key,item in prompt_original['single_select'].items()]
m_attributes = [key for key,item in prompt_original['multiple_select'].items()]


In [2]:
import random
import numpy as np
import json
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm
from math import sqrt


txt2img_url = r'http://127.0.0.1:7860/sdapi/v1/txt2img'

In [3]:
prompt=prompt_original
def get_attribute(at_value):
    for typ,ite in prompt.items():
        if typ=='quantitative_value':
            continue
        for key, item in prompt[typ].items():
            if key=='weights'or key=='multi_num':
                continue
            for j in range(len(prompt[typ][key])):
                if at_value in prompt[typ][key][j]:
                    return key
    return -1

get_attribute('angular line')

'line'

In [4]:
class Individual:
    

    def __init__(self,prompt,preferGenes,p_list):
        self.prompt = prompt
        self.preferGenes = preferGenes
        self.Gene_list = p_list
        self.dna = ['kandinsky',[],[],[],-1]
        
        #dna[1] for single attribute
        #dna[2] for multiple attribute
        #dna[3] for quantitative value
        #dna[4] for seed
        
        self.muteRate=0.05
        self.single_pick=1
        self.multiple_pick=2
        
    def quant_pick(self):
        brightness_input = self.prompt['quantitative_value']['brightness']
        parallel_input = self.prompt['quantitative_value']['parallel']
        structure_input = self.prompt['quantitative_value']['structure']
        
        #len==0: Choosing from user never happens
        if len(brightness_input)==0:
            b = np.random.uniform(-1,1)
        else:
            value = np.random.normal(np.mean(brightness_input), np.std(brightness_input))
            b = np.clip(value, -1, 1)
            
        if len(parallel_input)==0:
            p = np.random.uniform(-0.5,0.5)
        else:
            value = np.random.normal(np.mean(parallel_input), np.std(parallel_input))
            p = np.clip(value, -0.5, 0.5)
        if len(structure_input)==0:
            s = np.random.uniform(-0.5,0.5)
        else:
            value = np.random.normal(np.mean(structure_input),np.std(structure_input))
            s = np.clip(value,-1,1)
        return b,p,s
        
    def crossover(self,dna1,dna2):
        parents =[dna1,dna2]
        p_dict = {'sin':{},'mul':{}}
        at_list = []
        for i in parents:
            for j in i[1]:
                at = get_attribute(j)
                if at not in at_list:
                    p_dict['sin'][at] = j
                    at_list.append(at)
                else:
                    coin = random.randint(0,1)
                    if coin==0:
                        p_dict['sin'][at]=j
                        
            for j in i[2]:
                at = get_attribute(j[0])
                if at not in at_list:
                    at_list.append(at)
                    p_dict['mul'][at] = [m for m in j]
                else:
                    coin = random.randint(0,1)
                    if coin==0:
                        for m in j:
                            if m not in p_dict['mul'][at]:
                                p_dict['mul'][at].append(m)
                
                crossover_value = [v for v,w in self.prompt['multiple_select'][at]]
                crossover_weight =[w for v,w in self.prompt['multiple_select'][at]]
                index=[]
                for n in range(len(crossover_value)):
                    if crossover_value[n] in p_dict['mul'][at]:
                        index.append(n)
                crossover_weight=[crossover_weight[n] for n in index]
                p_dict['mul'][at]=list(np.random.choice(np.array(p_dict['mul'][at]),
                                                        size=2,replace=False,
                                                        p=np.array(crossover_weight)/sum(crossover_weight)))

        p_single_dna=[item for key, item in p_dict['sin'].items()]
        p_multiple_dna=[item for key, item in p_dict['mul'].items()]
        w_s = [self.prompt['single_select']['weights'][get_attribute(attr)] for attr in p_single_dna]
        w_m = [self.prompt['multiple_select']['weights'][get_attribute(attr[0])] for attr in p_multiple_dna]
        l1=list(np.random.choice(np.array(p_single_dna),
                                 size=self.single_pick,
                                 replace=False,
                                 p=np.array(w_s)/sum(w_s)))
        l2=list(np.random.choice(np.array([p for p in range(len(p_multiple_dna))]),
                                 size=self.multiple_pick,
                                 replace=False,
                                 p=np.array(w_m)/sum(w_m)))
        l2 = [p_multiple_dna[p] for p in l2]
        
        coin = random.randint(0,1)
        newseed=parents[coin][4]
        
        newbrightness=(parents[0][3][0]+parents[1][3][0])/2
        newdirection=(parents[0][3][1]+parents[1][3][1])/2
        newstructure=(parents[0][3][2]+parents[1][3][2])/2
        
        return l1,l2,[newbrightness,newdirection,newstructure],newseed
        
    def mutation(self,muteRate,single_dna,multiple_dna,newquant,newseed):
        #level1:mutation of attributes
        parents =[self.dna,['',single_dna,multiple_dna,newquant,newseed]]
        p_single_dna = []
        p_multiple_dna = []
        p_dict = {'sin':{},'mul':{}}
        at_list = []
        for i in parents:
            for j in i[1]:
                at = get_attribute(j)
                if at not in at_list:
                    p_dict['sin'][at] = j
                    at_list.append(at)
                else:
                    s=np.random.uniform(0,1)
                    if s<muteRate:
                        p_dict['sin'][at]=j
               
                #level2:mutation of values
                s=np.random.uniform(0,1)
                if s<muteRate:
                    items=self.prompt['single_select'][at]
                    weights=[q[1] for q in items]
                    values =[q[0] for q in items]
                    p_dict['sin'][at]= random.choices(values,weights=weights,k=1)[0]
            for j in i[2]:
                at = get_attribute(j[0])
                
                
                if at not in at_list:
                    at_list.append(at)
                    p_dict['mul'][at]=j
                else:
                    s=np.random.uniform(0,1)
                    if s<muteRate:
                        p_dict['mul'][at]=j
                
                #level2:mutation of values
                s=np.random.uniform(0,1)
                if s<muteRate:
                    items=self.prompt['multiple_select'][at]
                    weights=[q[1] for q in items]
                    values =[q[0] for q in items]
                    p_dict['mul'][at]= list(np.random.choice(np.array(values), size=3,replace=False,p=np.array(weights)/sum(weights)))
                    
        p_single_dna=[item for key, item in p_dict['sin'].items()]
        p_multiple_dna=[item for key, item in p_dict['mul'].items()]
        
        #print(p_single_dna,p_multiple_dna)
        w_s = [self.prompt['single_select']['weights'][get_attribute(attr)] for attr in p_single_dna]
        w_m = [self.prompt['multiple_select']['weights'][get_attribute(attr[0])] for attr in p_multiple_dna]
        l1=list(np.random.choice(np.array(p_single_dna),
                                 size=self.single_pick,
                                 replace=False,
                                 p=np.array(w_s)/sum(w_s)))
        l2=list(np.random.choice(np.array([p for p in range(len(p_multiple_dna))]),
                                 size=self.multiple_pick,
                                 replace=False,
                                 p=np.array(w_m)/sum(w_m)))
        l2 = [p_multiple_dna[p] for p in l2]
        
        s=np.random.uniform(0,1)
        if s<muteRate:
            muteseed=newseed
        else:
            muteseed=self.dna[4]
            
        s=np.random.uniform(0,1)
        if s<muteRate:
            mutequant=newquant
        else:
            mutequant=self.dna[3]
        
        return l1,l2,mutequant,muteseed
       
    
    def dna_generation(self):
        
        single_dna =[]
        multiple_dna=[]
        quant_dna=[]
        s = 0
        
        seed=random.randint(0, 2147483647)
        quant_dna = self.quant_pick()
        
        for typ,ite in self.prompt.items():
            if typ == 'quantitative_value':
                continue
            keys = []
            weights=[]
            
            for key, item in self.prompt[typ].items():
                if key=='weights'or key=='multi_num':
                    continue
                keys.append(key)
                w = self.prompt[typ]['weights'][key]
                weights.append(w)
            if(typ=='single_select'):
                s = self.single_pick
                # 在同一类型attributes中按照权重选取s个
                num = np.random.choice(np.array([i for i in range(len(self.prompt[typ])-1)]),
                                   size=s,
                                   replace=False,
                                   p=np.array(weights)/sum(weights))
            else:
                s = self.multiple_pick
                # 在同一类型attributes中按照权重选取s个
                num = np.random.choice(np.array([i for i in range(len(self.prompt[typ])-2)]),
                                   size=s,
                                   replace=False,
                                   p=np.array(weights)/sum(weights))
                
            
            for i in num:
                values = self.prompt[typ][keys[i]]
                value_weights = [j[1] for j in values]
                values = [j[0] for j in values]
                
                # 选取attribute对应的value
                if(typ=='single_select'):
                    selected_element = np.random.choice(np.array(values),
                                                        size=1,
                                                        replace=False,
                                                        p=np.array(value_weights)/sum(value_weights))
                    single_dna.append(selected_element[0])
                else:
                    #keys[i]: attribute name
                    selected_element = np.random.choice(np.array(values),
                                                        size=self.prompt[typ]['multi_num'][keys[i]],
                                                        replace=False,
                                                        p=np.array(value_weights)/sum(value_weights))
                    multiple_dna.append(list(selected_element))
        
        # 第一代
        if len(self.preferGenes)==0:
            self.dna[1]=single_dna
            self.dna[2]=multiple_dna
            self.dna[3]=quant_dna
            self.dna[4]=seed
            
        # 从上一代中，按照一定的规则和概率组合prompt (parent selection)
        else:
            a = random.choice(self.preferGenes)
            b = random.choice(self.preferGenes)
            a = self.Gene_list[int(a)]
            b = self.Gene_list[int(b)]
            self.dna[1],self.dna[2],self.dna[3],self.dna[4]=self.crossover(a,b)
            self.dna[1],self.dna[2],self.dna[3],self.dna[4]=self.mutation(self.muteRate,single_dna,multiple_dna,quant_dna,seed)


In [5]:
lora_kandinsky = "lora:kandinsky_t7t2_2:1"
lora_brightness = "lora:kandinsky_brightness_t"
lora_direction1 = "lora:direction_in"
lora_direction2 = "lora:direction_ex"
lora_structure = "lora:structure_v5"
#sort(by weights) and join the attributes
def dna2prompt(dna):
    at_list = []
    lora=[]
    for i in range(len(dna)):
        if i==0:
            at_list.append(dna[i])
        elif i==1:
            for j in dna[1]:
                at_list.append(j)
        elif i==2:
            for j in range(len(dna[2])):
                for q in dna[2][j]:
                    at_list.append(q)
                    
    #lora for kandinsky
    lora.append(lora_kandinsky)
    #lora for brightness
    lora.append(lora_brightness+':'+str(dna[3][0]))
    #lora for direction
    if dna[3][1]>0:
        lora.append(lora_direction1+':'+str(dna[3][1]))
    elif dna[3][1]<0:
        lora.append(lora_direction2+':'+str(-dna[3][1]))
    lora.append(lora_structure+':'+str(dna[3][2]))
        
    #at_list=sort_attribute(at_list)
    for item in lora:
        at_list.append('<'+item+'>')
    return ",".join(at_list)                 
                        
    

def submit_post(url: str, data: dict):
    return requests.post(url, data=json.dumps(data))

def save_encoded_image(b64_image: str, output_path: str):
    with open(output_path, 'wb') as image_file:
        image_file.write(base64.b64decode(b64_image))

def show_image(b64_image: bytes):
    bytesIO = BytesIO()
    img = Image.open(BytesIO(base64.b64decode(b64_image)))
    return img
        
# Validate user rating
def get_user_preference(n = 16):
    return input(f"输入你最喜欢的图片, 最少1张最多{n}张, 用空格分隔: ").split(" ")



def get_weight(key_value):
    
    for typ,ite in prompt.items():
        for key, item in prompt[typ].items():
            if key=='weights':
                continue
            for j in range(len(prompt[typ][key])):
                if key_value in prompt[typ][key][j]:
                    return prompt[typ][key][j][1]
    return -1

def get_attribute_weight(key_value):
    attribute= get_attribute(key_value)
    if attribute in s_attributes:
        return prompt['single_select']['weights'][attribute]
    elif attribute in m_attributes:
        return prompt['multiple_select']['weights'][attribute]
    else:
        return -1


def sort_attribute(attribute_list):
    temp=0
    for i in range(len(attribute_list)):
        weight = get_attribute_weight(attribute_list[i])
        if weight==-1:
            continue
        for j in range(1,i):
            if get_attribute_weight(attribute_list[j])<weight:
                attribute_list=swap_attribute(attribute_list,i,j)
    return attribute_list
        
def swap_attribute(a_list,p1,p2):
    a_list[p1],a_list[p2]=a_list[p2],a_list[p1]
    return a_list



In [6]:
import keyboard
batchsize = 16
prompt = prompt_original

dna_list = []
preferGenes = []
new_dna_list=[]
prompt_list=[]

def votes(i):
    delta=0.5
    print("vote for",i+1)
    d = dna_list[i]
    prompt['quantitative_value']['brightness'].append(d[3][0])
    prompt['quantitative_value']['parallel'].append(d[3][1])
    prompt['quantitative_value']['structure'].append(d[3][2])
    if i not in preferGenes:
        preferGenes.append(i)
    p = dna2prompt(d)
    print(p)
    p = p.split(",")[1:-1]
    for typ,ite in prompt.items():
        if typ=='quantitative_value':
            continue
        for key, item in prompt[typ].items():
            if key=='weights'or key=='multi_num':
                continue
            for j in range(len(prompt[typ][key])):
                word = prompt[typ][key][j][0]
                if word in p:
                    #prompt[typ]['weights'][key]+=1
                    prompt[typ][key][j][1] += delta
                    break



keyboard.add_hotkey('1', votes, args=(0,))
keyboard.add_hotkey('2', votes, args=(1,))
keyboard.add_hotkey('3', votes, args=(2,))
keyboard.add_hotkey('4', votes, args=(3,))
keyboard.add_hotkey('5', votes, args=(4,))
keyboard.add_hotkey('6', votes, args=(5,))
keyboard.add_hotkey('7', votes, args=(6,))
keyboard.add_hotkey('8', votes, args=(7,))
keyboard.add_hotkey('9', votes, args=(8,))
keyboard.add_hotkey('0', votes, args=(9,))
keyboard.add_hotkey('a', votes, args=(10,))
keyboard.add_hotkey('b', votes, args=(11,))
keyboard.add_hotkey('c', votes, args=(12,))
keyboard.add_hotkey('d', votes, args=(13,))
keyboard.add_hotkey('f', votes, args=(14,))
keyboard.add_hotkey('g', votes, args=(15,))
#recorded = keyboard.record(until='esc')

<function keyboard.add_hotkey.<locals>.remove_()>

In [ ]:
while(1):
    # 创建一个4x4的子图网格12345
    fig, axes = plt.subplots(int(sqrt(batchsize)), int(sqrt(batchsize)), figsize=(12, 12))
    new_dna_list=[]
    prompt_list=[]
    seed_list=[]
    for i, ax in tqdm(enumerate(axes.flat)):
        x = Individual(prompt,preferGenes,dna_list)
        x.dna_generation()
        new_dna_list.append(x.dna)
        p = dna2prompt(x.dna)
        prompt_list.append(p)
        seed_list.append(x.dna[4])
        data = {
                    'prompt': p,
                    'negative_prompt': '',
                    'sampler_index': 'Euler a',
                    'seed': x.dna[4],
                    'steps': 20,
                    'width': 512,
                    'height': 512,
                    'cfg_scale': 8
               }
        response = submit_post(txt2img_url, data)
        img = show_image(response.json()['images'][0])    
        ax.imshow(img)
        ax.axis('off')  # 不显示坐标轴
        title=str()
        if i>8:
            if i==9:
                title="0"
            elif i==10:
                title="a"
            elif i==11:
                title="b"
            elif i==12:
                title="c"
            elif i==13:
                title="d"
            elif i==14:
                title="f"
            elif i==15:
                title="g"
        else:
            title=i+1
        ax.set_title(title)
    dna_list = new_dna_list
    preferGenes=[]
        
    # 调整子图之间的间距
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    
    for i in range(batchsize):
        print(f"\nPrompt{i+1} :  ",prompt_list[i]," seed:",seed_list[i], end = "")
    print("")
    print("")
    print("To vote for them, press the corresponding keys above them.")
    # 展示子图
    plt.savefig("tmp.png")
    img = Image.open("tmp.png")
    img.show()
    
    # 为喜欢的图像投票
    # press the hotkeys:1~0,a~d,f~g
    
    if (input("\n你对生成的结果满意吗？如果不满意，可以继续生成图片。(yes/no) ") == "yes"):
        break
    
    #preferGenes = get_user_preference()
    #change_parameter(preferGenes, dna_list)


16it [01:16,  4.76s/it]



Prompt1 :   kandinsky,curve line,triangle,point,blue,orange,violet,<lora:kandinsky_t7t2_2:1>,<lora:kandinsky_brightness_t:-0.7226093238549147>,<lora:direction_ex:0.24224254096022257>,<lora:structure_v5:-0.07439324984609053>  seed: 411502525
Prompt2 :   kandinsky,curve line,white,green,red,triangle,square,<lora:kandinsky_t7t2_2:1>,<lora:kandinsky_brightness_t:-0.46789679469570666>,<lora:direction_in:0.3404064153446572>,<lora:structure_v5:-0.1953897619339412>  seed: 1567668430
Prompt3 :   kandinsky,straight line,blue,violet,green,square,point,<lora:kandinsky_t7t2_2:1>,<lora:kandinsky_brightness_t:-0.6540585686910572>,<lora:direction_ex:0.04447411385950162>,<lora:structure_v5:0.31925519919099754>  seed: 508605067
Prompt4 :   kandinsky,angular line,yellow,white,grey,triangle,circle,<lora:kandinsky_t7t2_2:1>,<lora:kandinsky_brightness_t:-0.9869226409180152>,<lora:direction_ex:0.20211879206780303>,<lora:structure_v5:0.3194056011713259>  seed: 195679154
Prompt5 :   kandinsky,curve line,circl